In [ ]:
script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using JLD
using Plots
# using ColorSchemes

In [3]:
### Plotting bettis




println("packages imported")

println("importing functions...")

include("helper_functions.jl")

println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)



### Set parameters

# Parameters for all graphs
const NNODES = 70
const MAXDIM = 3    # Maximum persistent homology dimension
const NREPS = 50


### Locate data
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("bettis.jld",x), readdir(read_dir))
println("Located the following graph files:")
for betti_file in betti_files
    println(betti_file)
end
model_names = [split(betti_file,"_")[1] for betti_file in betti_files]


### Create one large plot with all of the betti curves
nModels = length(betti_files)
nEdges = binomial(NNODES, 2)
bettisAll = zeros(NREPS, nEdges, MAXDIM, nModels)
for (i,betti_file) in enumerate(betti_files)

    # Read in Betti curves
    betti_dict = load("$(read_dir)/$(betti_file)")
    bettisAll[:,:,:, i] = betti_dict["bettisArray"]

end

bettisAllAvg = dropdims(mean(bettisAll, dims=1), dims=1)
println("Size bettisAllAvg is $(size(bettisAllAvg))")

# Finally plot
p1a = plot(bettisAllAvg[:,1,:], layout = 12, size = (1000,800), title = ["$(model_names[i])" for j=1:1, i=1:nModels])
for d=2:MAXDIM
    plot!(bettisAllAvg[:,d,:],legend = nothing, xlabel = "Edges added", ylabel="Beta_k")
    
end


savefig("../figures/all_bettis_test.pdf")





packages imported
importing functions...
packages and functions imported
Elapsed time = 125.15242099761963 seconds 
 
Located the following graph files:
DP_50_70_3_082520_graphs_forward_bettis.jld
IID_50_70_082520_graphs_forward_bettis.jld
RG_50_70_3_082520_graphs_forward_bettis.jld
RL_50_70_082520_graphs_forward_bettis.jld
assoc_700_100_100_20_20_09_05_082520_graphs_forward_bettis.jld
coreperiph_700_100_100_20_20_09_05_082520_graphs_forward_bettis.jld
cosineGeometric_50_70_3_082520_graphs_forward_bettis.jld
disassort_700_100_100_20_20_09_05_082520_graphs_forward_bettis.jld
discreteUniformConf_50_70_0_1000_082520_graphs_forward_bettis.jld
geometricConf_500_700_001_1000_082520_graphs_forward_bettis.jld
Size bettisAllAvg is (2415, 3, 10)
